# Prepare SpaceNet 7 Data for Model Training

We assume that initial steps of README have been executed and that this notebook is running in a docker container.  See the `src` directory for functions used in the algorithm.  

In [1]:
# Dataset location (edit as needed)
root_dir = '../'

In [2]:
import sys
print(sys.version)
!python --version

3.7.8 | packaged by conda-forge | (default, Nov 17 2020, 23:45:15) 
[GCC 7.5.0]
Python 3.7.8


In [3]:
from platform import python_version
print(python_version())

3.7.8


In [4]:
import multiprocessing
import pandas as pd
import numpy as np
import skimage
import gdal
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("__file__"))))
import matplotlib as mpl
import matplotlib.cm as cmx
import matplotlib.pyplot as plt
import matplotlib.colors as colors
plt.rcParams.update({'font.size': 16})
mpl.rcParams['figure.dpi'] = 300
import vector
# get_list = os.listdir(os.path.join('../data/train/AOI_5_Khartoum_Train/RGB-PanSharpen/'))
# print(len(get_list))
from raster.image import create_multiband_geotiff
from utils.core import _check_gdf_load

# import from data_prep_funcs
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.prep_funcs import map_wrapper, make_geojsons_and_masks

/home/hoyin/anaconda3/envs/solaris2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hoyin/anaconda3/envs/solaris2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hoyin/anaconda3/envs/solaris2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hoyin/anaconda3/envs/solaris2/lib/pyt

In [7]:
# Create Training Masks
# Multi-thread to increase speed
# We'll only make a 1-channel mask for now, but Solaris supports a multi-channel mask as well, see
#     https://github.com/CosmiQ/solaris/blob/master/docs/tutorials/notebooks/api_masks_tutorial.ipynb
mode = 'Train'
# mode = 'test'

aois = sorted([f for f in os.listdir(os.path.join(root_dir, 'data',mode))
               if os.path.isdir(os.path.join(root_dir, 'data',mode, f))])
# print(aois)
# # aois.remove("AOI_2_Vegas_"+mode)
aois.remove("AOI_3_Paris_Train")
aois.remove("AOI_4_Shanghai_Train")
aois.remove("AOI_5_Khartoum_Train")
n_threads = 10
params = [] 
make_fbc = True

input_args = []
for i, aoi in enumerate(aois):
    print(i, "aoi:", aoi)
    im_dir = os.path.join(root_dir, 'data', mode, aoi, 'RGB-PanSharpen/')
#     json_dir = os.path.join(root_dir,'data', 'test', aoi, 'geojson')
    json_dir = os.path.join(root_dir,'data', mode, aoi, 'geojson','buildings')
    out_dir_mask = os.path.join(root_dir, 'data', mode, aoi, 'masks/')
    out_dir_mask_fbc = os.path.join(root_dir, 'data', mode, aoi, 'masks_fbc/')
    os.makedirs(out_dir_mask, exist_ok=True)
    if make_fbc:
        os.makedirs(out_dir_mask_fbc, exist_ok=True)

    json_files = sorted([f
                for f in os.listdir(os.path.join(json_dir))
#                 if f.endswith('Buildings.geojson') and os.path.exists(os.path.join(json_dir, f))])
                if f.endswith('.geojson') and os.path.exists(os.path.join(json_dir, f))])
    for j, f in enumerate(json_files):
        # print(i, j, f)
        name_root = f.split('.')[0]
        json_path = os.path.join(json_dir, f)
#         image_path = os.path.join(im_dir, name_root + '.tif').replace('labels', 'images').replace('_Buildings', '')
        image_path = os.path.join(im_dir, name_root + '.tif').replace('buildings', 'RGB-PanSharpen')
        output_path_mask = os.path.join(out_dir_mask, name_root + '.tif')
        if make_fbc:
            output_path_mask_fbc = os.path.join(out_dir_mask_fbc, name_root + '.tif')
        else:
            output_path_mask_fbc = None
            
        if (os.path.exists(output_path_mask)):
             continue
        else: 
            input_args.append([make_geojsons_and_masks, 
                               name_root, image_path, json_path,
                               output_path_mask, output_path_mask_fbc])

# execute 
print("len input_args", len(input_args))
print("Execute...\n")
with multiprocessing.Pool(n_threads) as pool:
    pool.map(map_wrapper, input_args)


0 aoi: AOI_2_Vegas_Train


FileNotFoundError: [Errno 2] No such file or directory: '../data/Train/AOI_2_Vegas_Train/geojson/buildings'

In [ ]:
# Inspect visually

aoi = 'AOI_2_Vegas_Train'
im_dir = os.path.join(root_dir, 'data', mode, aoi, 'RGB-PanSharpen')
mask_dir = os.path.join(root_dir, 'data', mode, aoi, 'masks')

im_list = sorted([z for z in os.listdir(im_dir) if z.endswith('.tif')])
im_file = im_list[28]

im_path = os.path.join(im_dir, im_file)
mask_path = os.path.join(mask_dir, im_file.split('.')[0] + '.tif').replace('RGB-PanSharpen','buildings')
im = skimage.io.imread(im_path)
mask = skimage.io.imread(mask_path)
print(np.max(mask))
print(np.min(mask))
print(type(mask))
boundary = skimage.segmentation.find_boundaries(mask, mode='subpixel', background=0).astype(np.float32)
print(type(boundary))
print(np.max(boundary))
print(np.min(boundary))
figsize=(24, 12)
# name = im_file.split('.')[0].split('global_monthly_')[-1]
name = im_file.split('.')[0]
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=figsize)

_ = ax0.imshow(mask)
ax0.set_xticks([])
ax0.set_yticks([])
_ = ax0.set_title(name)

_ = ax1.imshow(boundary)
ax1.set_xticks([])
ax1.set_yticks([])
_ = ax1.set_title(name)
_ = fig.suptitle(name)
plt.tight_layout()

In [ ]:
# make identifier plots

aoi = 'AOI_3_Paris_Test'
im_dir = os.path.join(root_dir,'data', 'test', aoi, 'RGB-PanSharpen')
# json_dir = os.path.join(root_dir, 'data','test', aoi, 'geojson', 'buildings')
json_dir = os.path.join(root_dir, 'data','test', aoi, 'geojson')

# colors
vmax = 200
cmap = plt.get_cmap('hsv')  # 'jet'
cNorm  = colors.Normalize(vmin=0, vmax=vmax)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmap)
plot_only_first_and_last = True
label_font_size = 4
im_pix_size_x, im_pix_size_y = 650, 650
figsize=(20,20)
# print(json_dir)
json_files = sorted([f
            for f in os.listdir(json_dir) 
            if f.endswith('.geojson') and os.path.exists(os.path.join(json_dir, f))])
# print(json_files)
if plot_only_first_and_last:
    json_files = [json_files[0], json_files[-1]]
#     json_files = [json_files[1]]

# convert json
for j, f in enumerate(json_files):
    # print(i, j, f)
    name_root = f.split('.')[0]
    json_path = os.path.join(json_dir, f)
    image_path = os.path.join(im_dir, name_root + '.tif').replace('buildings', 'RGB-PanSharpen')
    print("name_root:", name_root)
    # print("json_path", json_path)
    # output_path = os.path.join(out_dir, f)
    # if os.path.exists(output_path):
    #    print("  path exists, skipping:", name_root)
    #    continue
    gdf_pix = vector.polygon.geojson_to_px_gdf(json_path, image_path, geom_col='geometry', precision=None, output_path=None, override_crs=False)
    fig, ax = plt.subplots(figsize=figsize)
    for _, row in gdf_pix.iterrows():
        geom = row['geometry']
        poly_id = row['Id']
        x, y = geom.exterior.xy
        cx, cy = np.array(geom.centroid.xy).astype(float)
        #print("centroid:", centroid)
        color_int = int(poly_id) % vmax
        colorVal = scalarMap.to_rgba(color_int)
        ax.plot(x, y, c=colorVal)
        # poly id
        ax.annotate(str(poly_id), xy=(cx, cy), ha='center', size=label_font_size)
        #text_object = plt.annotate(label, xy=(x_values[i], y_values[i]), ha='center')
        #ax.text(cx, cy, str(poly_id))
    ax.set_xlim(0, im_pix_size_x)
    ax.set_ylim(0, im_pix_size_y)
    title = str(j) + " - " + name_root + " - N buildings = " + str(len(gdf_pix))
    ax.set_title(title)

    plt.show()

In [6]:
# Make dataframe csvs for train/test

out_dir = os.path.join(root_dir, 'csvs/')
os.makedirs(out_dir, exist_ok=True)
d = os.path.join(root_dir, 'data', mode, 'AOI_2_Vegas_Train')
subdirs = sorted([f for f in os.listdir(d)])    
outpath = os.path.join(out_dir, 'solaris_unet_' + mode + '_df.csv')
im_list, mask_list = [], []


im_files = [os.path.join( d,'RGB-PanSharpen',  f.split('.')[0] + '.tif')
for f in sorted(os.listdir(os.path.join(d,'RGB-PanSharpen' ).replace('masks','RGB-PanSharpen')))]

mask_files = [os.path.join(d, 'masks', f.split('.')[0] + '.tif')
for f in sorted(os.listdir(os.path.join(d, 'masks').replace('RGB-PanSharpen','masks')))]


im_list.extend(im_files)
mask_list.extend(mask_files)
    

df = pd.DataFrame({'image': im_list, 'label': mask_list})
display(df.head())

df.to_csv(outpath, index=False)
print(mode, "len df:", len(df))
print("output csv:", outpath)

FileNotFoundError: [Errno 2] No such file or directory: '../data/train/AOI_2_Vegas_Train'